# System building: Protein with Ligand

In this tutorial, we will showcase how to build a protein-ligand system for simulating ligand binding. The sample system is Trypsin (the protein) and benzamidine (the ligand).

Let's start with some helpful imports and definitions:

In [1]:
from htmd.ui import *
from htmd.home import home
from os.path import join
from moleculekit.config import config

config(viewer='webgl')
datadir = home(dataDir='building-protein-ligand')

2024-06-11 14:58:30,793 - numexpr.utils - INFO - Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-06-11 14:58:30,794 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.
2024-06-11 14:58:30,900 - rdkit - INFO - Enabling RDKit 2022.09.1 jupyter extensions



Please cite HTMD: Doerr et al.(2016)JCTC,12,1845. https://dx.doi.org/10.1021/acs.jctc.6b00049
HTMD Documentation at: https://software.acellera.com/htmd/

You are on the latest HTMD version (2.3.28+0.g1f64e666d.dirty).



## Load the protein-ligand complex

One can obtain the protein-ligand complex from the PDB database (ID:3PTB).

In [2]:
# One can download it directly from the RCSB servers simply by providing the PDB code
prot = Molecule('3PTB')

In [3]:
prot.view()

NGLWidget()

## Clean the structures

The PDB crystal structure contains the protein as well as water molecules, a calcium ion and a ligand. Here we will start by removing the ligand from the protein Molecule as we will add it later to manipulate it separately.

In [4]:
prot.remove('resname BEN')

2024-06-11 14:58:34,227 - moleculekit.molecule - INFO - Removed 9 atoms. 1692 atoms remaining in the molecule.


array([1630, 1631, 1632, 1633, 1634, 1635, 1636, 1637, 1638], dtype=int32)

## Preparing the protein

In this step, we prepare the protein for simulation by adding hydrogens, setting the protonation states, and optimizing the protein (more details on the protein preparation tutorial):

In [5]:
prot = systemPrepare(prot, pH=7.4)

2024-06-11 14:58:34,332 - moleculekit.tools.preparation - WARNING - Both chains and segments are defined in Molecule.chain / Molecule.segid, however they are inconsistent. Protein preparation will use the chain information.



---- Molecule chain report ----
Chain A:
    First residue: ILE    16  
    Final residue: HOH   809  
---- End of chain report ----



2024-06-11 14:58:35,888 - moleculekit.tools.preparation - WARNING - The following residues have not been optimized: CA
2024-06-11 14:58:35,968 - moleculekit.tools.preparation - INFO - Modified residue CYS    22 A to CYX
2024-06-11 14:58:35,969 - moleculekit.tools.preparation - INFO - Modified residue HIS    40 A to HIE
2024-06-11 14:58:35,969 - moleculekit.tools.preparation - INFO - Modified residue CYS    42 A to CYX
2024-06-11 14:58:35,970 - moleculekit.tools.preparation - INFO - Modified residue HIS    57 A to HIP
2024-06-11 14:58:35,970 - moleculekit.tools.preparation - INFO - Modified residue CYS    58 A to CYX
2024-06-11 14:58:35,971 - moleculekit.tools.preparation - INFO - Modified residue HIS    91 A to HID
2024-06-11 14:58:35,971 - moleculekit.tools.preparation - INFO - Modified residue CYS   128 A to CYX
2024-06-11 14:58:35,971 - moleculekit.tools.preparation - INFO - Modified residue CYS   136 A to CYX
2024-06-11 14:58:35,972 - moleculekit.tools.preparation - INFO - Modified

## Define segments

To build a system in HTMD, we need to separate the chemical molecules into separate segments. This prevents the builder from accidentally bonding different chemical molecules and allows us to add caps to them.

In [6]:
prot = autoSegment(prot, sel='protein')
prot.set('segid', 'W', sel='water')
prot.set('segid', 'CA', sel='resname CA')

2024-06-11 14:58:36,020 - moleculekit.tools.autosegment - INFO - Created segment P0 between resid 16 and 245.


Center the protein to the origin

In [7]:
prot.center()

## Let's work on the ligand!

Load the ligand from the HTMD data directory. Normally to perform this step you will first need to parameterize your ligand for the AMBER forcefield with a tool like Parameterize (available on [PlayMolecule](open.playmolecule.org)) which produces a topology (cif/mol2) and a parameter (frcmod) file. Here for the sake of the tutorial we provide both files in your HTMD installation.

In [8]:
ligand = Molecule(join(datadir, 'BEN.cif'))

Let's center the ligand and visualize it:

In [9]:
ligand.center()
ligand.view()

NGLWidget()

In [10]:
# We can give a convenient segid and resname to the ligand
# The resname should be BEN to match the parameters in the
# rtf and prm files.
ligand.set('segid','L')
ligand.set('resname','BEN')

But the ligand is now located inside the protein...
We would like the ligand to be:

* At a certain distance from the protein
* Rotated randomly, to provide different starting conditions

## Let's randomize the ligand position

In [11]:
ligand.rotateBy(uniformRandomRotation())

This took care of the ligand rotation around its own center. 
We still need to position it far from the protein.
First, find out the radius of the protein:

![maxdist](img/maxdist.png)

In [12]:
from moleculekit.util import maxDistance
D = maxDistance(prot, 'all')
print(D)

28.830551


In [13]:
D += 10
# Move the ligand 10 Angstrom away from the furthest protein atom in X dimension
ligand.moveBy([D, 0, 0])  
# rotateBy rotates by default around [0, 0, 0]. Since the ligand has been moved
# away from the center it will be rotated in a sphere of radius D+10 around [0, 0, 0]
ligand.rotateBy(uniformRandomRotation())

### Mix it all together

In [14]:
mol = Molecule(name='combo')
mol.append(prot)
mol.append(ligand)
mol.reps.add(sel='protein', style='NewCartoon', color='Secondary Structure')
mol.reps.add(sel='resname BEN', style='Licorice')
mol.view()

NGLWidget()

## Solvate

> Water is the driving force of all nature. --Leonardo da Vinci

![waterbox](http://pub.htmd.org/tutorials/system-building-protein-ligand/waterbox.png)

In [15]:
# We solvate with a larger cubic box to fully solvate the ligand
DW = D + 5
smol = solvate(mol, minmax=[[-DW, -DW, -DW], [DW, DW, DW]])
smol.reps.add(sel='water', style='Lines')
smol.view()

2024-06-11 14:58:36,251 - htmd.builder.solvate - INFO - Using water pdb file at: /home/sdoerr/Work/htmd/htmd/share/solvate/wat.pdb
2024-06-11 14:58:36,587 - htmd.builder.solvate - INFO - Replicating 8 water segments, 2 by 2 by 2
Solvating: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:02<00:00,  3.05it/s]
2024-06-11 14:58:39,744 - htmd.builder.solvate - INFO - 20145 water molecules were added to the system.


NGLWidget()

## Build the System with a specific forcefield

HTMD aims to be force-field agnostic. After you have a built system, you can either build it in Amber or CHARMM. The following sections work on the same previously solvated system and can be interconverted. However due to licensing issues we are not able to provide CHARMM support and recommend using the AMBER forcefield.

Special care must be taken care in this case due to the use of benzamidine, which is not present by default on the respective forcefields.

### CHARMM forcefield

In [16]:
charmm.listFiles()

---- Topologies files list: /home/sdoerr/Work/htmd/htmd/share/builder/charmmfiles/top/ ----
/top/top_all22_prot.rtf
/top/top_all22star_prot.rtf
/top/top_all35_ethers.rtf
/top/top_all36_carb.rtf
/top/top_all36_cgenff.rtf
/top/top_all36_lipid.rtf
/top/top_all36_lipid_ljpme.rtf
/top/top_all36_na.rtf
/top/top_all36_prot.rtf
/top/top_water_ions.rtf
---- Parameters files list: /home/sdoerr/Work/htmd/htmd/share/builder/charmmfiles/par/ ----
/par/par_all22_prot.prm
/par/par_all22star_prot.prm
/par/par_all35_ethers.prm
/par/par_all36_carb.prm
/par/par_all36_cgenff.prm
/par/par_all36_lipid.prm
/par/par_all36_lipid_ljpme.prm
/par/par_all36_na.prm
/par/par_all36m_prot.prm
/par/par_water_ions.prm
---- Stream files list: /home/sdoerr/Work/htmd/htmd/share/builder/charmmfiles/str/ ----
/str/carb/toppar_all36_carb_glycolipid.str
/str/carb/toppar_all36_carb_glycopeptide.str
/str/carb/toppar_all36_carb_imlab.str
/str/carb/toppar_all36_carb_lignin.str
/str/carb/toppar_all36_carb_model.str
/str/cphmd/protp

### Build and ionize using CHARMM

In [17]:
topos_charmm  = charmm.defaultTopo() + [join(datadir, 'BEN.rtf')]
params_charmm = charmm.defaultParam() + [join(datadir, 'BEN.prm')]

# bmol_charmm = charmm.build(smol, topo=topos_charmm, param=params_charmm, outdir='./build_charmm')

### AMBER forcefield

In [18]:
amber.listFiles()

---- Forcefield files list: /home/sdoerr/miniforge3/envs/htmd/dat/leap/cmd/ ----
leaprc.amberdyes
leaprc.conste
leaprc.constph
leaprc.DNA.bsc1
leaprc.DNA.OL15
leaprc.ffAM1
leaprc.ffPM3
leaprc.fluorine.ff15ipq
leaprc.gaff
leaprc.gaff2
leaprc.GLYCAM_06EPb
leaprc.GLYCAM_06j-1
leaprc.lipid21
leaprc.mimetic.ff15ipq
leaprc.modrna08
leaprc.music
leaprc.phosaa10
leaprc.phosaa14SB
leaprc.phosaa19SB
leaprc.protein.fb15
leaprc.protein.ff03.r1
leaprc.protein.ff03ua
leaprc.protein.ff14SB
leaprc.protein.ff14SB_modAA
leaprc.protein.ff14SBonlysc
leaprc.protein.ff15ipq
leaprc.protein.ff15ipq-vac
leaprc.protein.ff19ipq
leaprc.protein.ff19SB
leaprc.protein.ff19SB_modAA
leaprc.RNA.LJbb
leaprc.RNA.OL3
leaprc.RNA.ROC
leaprc.RNA.Shaw
leaprc.RNA.YIL
leaprc.water.fb3
leaprc.water.fb4
leaprc.water.opc
leaprc.water.opc3
leaprc.water.opc3pol
leaprc.water.spce
leaprc.water.spceb
leaprc.water.tip3p
leaprc.water.tip4pd
leaprc.water.tip4pd-a99SBdisp
leaprc.water.tip4pew
leaprc.xFPchromophores
---- OLD Forcefield file

### Build and ionize using Amber

In [19]:
topos_amber = amber.defaultTopo() + [join(datadir, 'BEN.cif')]
frcmods_amber = amber.defaultParam() + [join(datadir, 'BEN.frcmod')]

In [20]:
bmol_amber = amber.build(smol, topo=topos_amber, param=frcmods_amber, outdir='./build_amber', saltconc=0.15)

2024-06-11 14:58:44,189 - htmd.builder.amber - INFO - Detecting disulfide bonds.
2024-06-11 14:58:44,197 - htmd.builder.builder - INFO - 6 disulfide bonds were added


Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 26, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 42, insertion: '', segid: 'P0'>

Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 117, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 184, insertion: '', segid: 'P0'>

Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 174, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 198, insertion: '', segid: 'P0'>

Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 149, insertion: '', segid: 'P0'>
                   and: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 163, insertion: '', segid: 'P0'>

Disulfide Bond between: UniqueResidueID<resname: 'CYX', chain: 'A', resid: 8, insertion: '', segid: 'P0'>
                   and: Uniq

2024-06-11 14:58:45,604 - htmd.builder.amber - INFO - Starting the build.
2024-06-11 14:58:48,790 - htmd.builder.amber - INFO - Finished building.
2024-06-11 14:58:49,566 - moleculekit.writers - WARNING - Field "resid" of PDB overflows. Your data will be truncated to 4 characters.
2024-06-11 14:58:54,542 - htmd.builder.ionize - INFO - Adding 10 anions + 0 cations for neutralizing and 0 ions for the given salt concentration 0 M.
2024-06-11 14:58:57,279 - htmd.builder.amber - INFO - Starting the build.
2024-06-11 14:59:00,594 - htmd.builder.amber - INFO - Finished building.
2024-06-11 14:59:01,416 - moleculekit.writers - WARNING - Field "resid" of PDB overflows. Your data will be truncated to 4 characters.
2024-06-11 14:59:05,984 - py.warnings - WARNING - /home/sdoerr/miniforge3/envs/htmd/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please 

## Visualize

The built system can be visualized (with waters hidden to be able to visualize the inserted ions):

In [21]:
bmol_amber.view(sel='not water') # visualize the charmm built system
# bmol_amber.view(sel='not water') # uncomment to visualize the amber built system

2024-06-11 14:59:09,148 - moleculekit.writers - WARNING - Field "resid" of PDB overflows. Your data will be truncated to 4 characters.


NGLWidget()

The `bmol_charmm` and `bmol_amber` are `Molecule` objects that contain the built system, but the full contents to run a simulation are located in the `outdir` (`./build` in this case).